# Tutorial on how to use mescal with the PathwayESM class

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Import the required libraries
from mescal import *
import pandas as pd
import bw2data as bd

## Set up and input data

In [ ]:
ecoinvent_version = '3.10.1' # choose the ecoinvent version you wish to use
esm_location = 'CH' # choose the version of energyscope for which you want to generate metrics
spatialized_database = True # set to True if you want to use your spatialized version of ecoinvent
regionalize_foregrounds = ['Operation'] # set to 'all' if you want to regionalize the foreground inventories of all types of LCI datasets
premise_iam = 'image' # choose the IAM to which the premise database is linked
premise_ssp_rcp = 'SSP2-Base' # choose the SSP/RCP scenario to which the premise database is linked

In [ ]:
# Set the name of your main LCI database (e.g., ecoinvent or premise database) here:
name_premise_db_2020 = f"ecoinvent_cutoff_{ecoinvent_version}_{premise_iam}_{premise_ssp_rcp}_2020+truck_carculator regionalized_wo"
name_premise_db_2050 = f"ecoinvent_cutoff_{ecoinvent_version}_{premise_iam}_{premise_ssp_rcp}_2050+truck_carculator regionalized_wo"

if spatialized_database:
    name_spatialized_biosphere_db = 'biosphere3_spatialized_flows'
else:
    name_spatialized_biosphere_db = None

In [ ]:
# Set the general name of the new database with the ESM results (the different years will be added to that name)
esm_db_name = f'EnergyScope_Pathway_{esm_location}'

In [ ]:
# Main version of ecoinvent (without .1 if any)
ecoinvent_main_version = '.'.join(ecoinvent_version.split('.')[:2])

In [ ]:
lcia_methods=[
    f'IMPACT World+ Midpoint 2.1 for ecoinvent v{ecoinvent_main_version}',
    f'IMPACT World+ Damage 2.1 for ecoinvent v{ecoinvent_main_version}',
    f'IMPACT World+ Footprint 2.1 for ecoinvent v{ecoinvent_main_version}',
]

In [ ]:
# Set up your Brightway project
bd.projects.set_current(f'ecoinvent{ecoinvent_version}') # put the name of your brightway project here

In [ ]:
premise_db_2020 = Database(name_premise_db_2020, create_pickle=True)
premise_db_2050 = Database(name_premise_db_2050, create_pickle=True)

In [ ]:
if regionalize_foregrounds is not None and spatialized_database:
    spatialized_biosphere_db = Database(name_spatialized_biosphere_db)
else:
    spatialized_biosphere_db = None

In [ ]:
# Inputs
# Mandatory
mapping = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}.csv')
unit_conversion = pd.read_excel(f'../dev/energyscope_data/{esm_location}/unit_conversion_{ecoinvent_version}.xlsx')
mapping_esm_flows_to_CPC = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_esm_flows_to_CPC.csv')
model = pd.read_csv(f'../dev/energyscope_data/{esm_location}/model.csv')
impact_abbrev = pd.read_csv('../dev/lcia/impact_abbrev.csv')

# Optional
technology_compositions = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technology_compositions.csv')
technology_specifics = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technology_specifics.csv')
lifetime = pd.read_csv(f'../dev/energyscope_data/{esm_location}/lifetime.csv')
efficiency = pd.read_csv(f'../dev/energyscope_data/{esm_location}/efficiency.csv')
mapping_product_to_CPC = pd.read_csv('../mescal/data/mapping_new_products_to_CPC.csv')
technologies_to_remove_from_layers = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technologies_to_remove_from_layers.csv')
new_end_use_types = pd.read_csv(f'../dev/energyscope_data/{esm_location}/new_end_use_types.csv')
results_from_esm = pd.read_csv(f'../dev/energyscope_data/{esm_location}/results_ES.csv')

In [ ]:
# The database name in the mapping file should be the one of the first time step
mapping.Database = name_premise_db_2020

In [ ]:
# sufficient match within ecoinvent
if esm_location == 'CA-QC':
    accepted_locations = ['CA-QC']
elif esm_location == 'CH':
    accepted_locations = ['CH']
else:
    accepted_locations = ['GLO', 'RoW']

In [ ]:
# Define the user-defined ranking
if esm_location == 'CA-QC':
    my_ranking = [
        'CA-QC', # Quebec
        'CA', # Canada
        'CAN', # Canada in IMAGE
        'CAZ', # Canada - Australia - New Zealand in REMIND
        'RNA', # North America
        'US', # United States
        'USA', # United States in REMIND and IMAGE
        'GLO', # Global average
        'RoW', # Rest of the world
    ]
elif esm_location == 'CH':
    my_ranking = [
        'CH',
        'NEU',
        'EUR',
        'WEU',
        'RER',
        'IAI Area, EU27 & EFTA',
        'GLO',
        'RoW'
    ]
else:
    my_ranking = [
        'GLO',
        'RoW',
    ]

In [ ]:
# If necessary, add missing CPC categories to the database
premise_db_2020.add_CPC_categories()
premise_db_2050.add_CPC_categories()

## Initialize the PathwayESM class

In [ ]:
esm = PathwayESM(
    # Mandatory inputs
    time_steps=[
        {'year': 2020, 'model': model, 'lifetime':lifetime, 'main_database': premise_db_2020},
        {'year': 2050, 'model': model, 'lifetime':lifetime, 'main_database': premise_db_2050},
    ],
    mapping=mapping,
    unit_conversion=unit_conversion,
    mapping_esm_flows_to_CPC_cat=mapping_esm_flows_to_CPC,
    esm_db_name=esm_db_name,
    biosphere_db_name='ecoinvent-3.10.1-biosphere',
    esm_location=esm_location,

    # Optional inputs
    technology_compositions=technology_compositions,
    tech_specifics=technology_specifics,
    efficiency=efficiency,
    regionalize_foregrounds=regionalize_foregrounds,
    accepted_locations=accepted_locations,
    locations_ranking=my_ranking,
    spatialized_biosphere_db=spatialized_biosphere_db,
    results_path_file=f'results/energyscope_pathway_{esm_location}/',
    stop_background_search_when_first_flow_found=True,
    esm_end_use_demands=['ELECTRICITY_EHV', 'HEAT_HIGH_T'],
    remove_double_counting_to=['Operation', 'Construction'],
)

In [ ]:
esm.check_inputs()

In [ ]:
missing_flows = premise_db_2020.test_mapping_file(esm.mapping[esm.mapping.Year == 2020])

## Create the ESM databases

In [ ]:
esm.create_esm_database()

## Compute the impact scores

In [ ]:
R_long, contrib_analysis_res, _ = esm.compute_impact_scores(
    methods=lcia_methods,
    contribution_analysis='both',
    contribution_analysis_limit_type='number',
    contribution_analysis_limit=3,
)
R_long.to_csv(f'{esm.results_path_file}/impact_scores.csv', index=False)
contrib_analysis_res.to_csv(f'{esm.results_path_file}/contribution_analysis.csv', index=False)

In [ ]:
R_long.head()

In [ ]:
contrib_analysis_res.head()

## Create the .dat and .mod files

In [ ]:
specific_lcia_abbrev = ['m_CCS', 'TTHH', 'TTEQ']

In [ ]:
# Create .dat file
esm.normalize_lca_metrics(
    R=R_long,
    mip_gap=1e-6,
    lcia_methods=lcia_methods,
    specific_lcia_abbrev=specific_lcia_abbrev,
    impact_abbrev=impact_abbrev,
    path=esm.results_path_file,
)

In [ ]:
# Create the .mod file
esm.generate_mod_file_ampl(
    lcia_methods=lcia_methods,
    impact_abbrev=impact_abbrev,
    specific_lcia_abbrev=specific_lcia_abbrev,
    path=esm.results_path_file,
)

## Feedback of ESM results in the LCI database

In [ ]:
esm.create_new_database_with_esm_results(  # creates a new database with the ESM results for 2020 and 2050
    esm_results=results_from_esm,
    new_end_use_types=new_end_use_types,
    tech_to_remove_layers=technologies_to_remove_from_layers,
    write_database=True,
    remove_background_construction_flows=False,
    harmonize_efficiency_with_esm=True,
    harmonize_capacity_factor_with_esm=True,
)

In [ ]:
new_db = esm.connect_esm_results_to_database(
    create_new_db=True,
    new_db_name='main_database_with_esm_results_from_2020',
)  # connects results of time step 1 (2020) to the database of time step 2 (2050)